In [84]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import subprocess
import time

In [7]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

url = f"https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%9D%98_%EC%8B%AC%EC%95%BC%EB%B2%84%EC%8A%A4_%EB%85%B8%EC%84%A0_%EB%AA%A9%EB%A1%9D"

result = requests.get(url, headers=headers)
soup = BeautifulSoup(result.text, "html.parser")  # 페이지 불러와서 bs4로 변경

k = soup.find("table", "wikitable")  # 페이지 별 최신 날짜 찾기

In [19]:
subprocess.Popen('C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="D:\chromdriver\Temp"')
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
chrome_driver = "D:\chromdriver\chromedriver.exe"
browser = webdriver.Chrome(chrome_driver, options=chrome_options)
browser.implicitly_wait(time_to_wait=7)

C:\Users\679oo\AppData\Local\Temp/ipykernel_5884/2094588.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_driver, options=chrome_options)


In [200]:
bus_names = [
    "N13",
    "N15",
    "N16",
    "N26",
    "N30",
    "N32",
    "N34",
    "N37",
    "N51",
    "N61",
    "N62",
    "N64",
    "N72",
    "N75",
]
result = []
for bus_name in bus_names:
    # 검색
    browser.get("https://bus.go.kr/searchResult6.jsp")
    searching = browser.find_element(By.ID, "searchname")
    searching.send_keys(f"{bus_name}")
    time.sleep(0.1)
    searching.send_keys(Keys.ENTER)
    time.sleep(0.1)

    # irfame 이동 후 scraping
    browser.switch_to.frame("introflash")
    html1 = BeautifulSoup(browser.page_source, "html.parser")
    st_id = [a.text for a in html1.find_all("text", "stationNo")]

    # dataframe만들기
    st_id_dt = pd.DataFrame(st_id, columns=["id"])
    st_id_dt["bus"] = f"{bus_name}"
    result.append(st_id_dt)


final = pd.concat(result, axis=0)

final


,id,bus
0,09104,N15
1,09286,N15
2,10342,N15
3,10184,N15
4,10186,N15
...,...,...
168,12013,N75
169,,N75
170,12111,N75
171,12009,N75


In [209]:
master_bus = pd.read_csv('assets/서울시 정류장마스터 정보.csv',encoding="CP949")

master_bus = master_bus.query('정류장_유형=="중앙차로"')

# remove_row = [48899,47206,47022,42811,38961,39246,39248,38326,38329,38244,38245,34215,34099,31924,31899,30741,30742,30811,30812]

master_bus.drop(index=remove_row,inplace=True)

In [210]:
final_copy = final.sort_values(by='id').iloc[21:]

final_copy['id']=final_copy['id'].astype(int)

In [211]:
night_bus_info=pd.merge(final_copy,master_bus,how='left',left_on='id',right_on='정류장_번호').drop(columns=['정류장_번호','버스도착정보안내기_설치_여부','정류장_ID','정류장_유형'])

In [212]:
night_bus_info

,id,bus,정류장_명칭,위도,경도
0,1001,N37,종로2가사거리,126.987750,37.569765
1,1004,N37,종로2가.삼일교,126.987613,37.568579
2,1006,N75,서대문역사거리,126.966893,37.566137
3,1006,N26,서대문역사거리,126.966893,37.566137
4,1007,N26,서울역사박물관.경희궁앞,126.970380,37.569135
...,...,...,...,...,...
1702,25349,N30,NaN,NaN,NaN
1703,35160,N37,NaN,NaN,NaN
1704,35331,N72,NaN,NaN,NaN
1705,38003,N61,현대모닝사이드1차아파트.새마을,127.154391,37.360647


In [165]:
a = master_bus['정류장_번호'].value_counts()
bus_left = a[a>1].index
# master_bus[]

In [163]:
night_bus_info['id'].value_counts()[:10]

22011    8
22103    6
1044     6
1041     5
13021    5
14061    5
1037     5
1023     5
6026     4
2173     4
Name: id, dtype: int64

In [196]:
# master_bus.query('정류장_번호	 == 2007')
master_bus.query('정류장_명칭	 == "오이도중앙로입구"')

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부


In [167]:


night_bus_info[night_bus_info.id.isin(bus_left)] 

,id,bus,정류장_명칭,위도,경도
70,2007,N15,경기도동물위생시험소,126.940002,37.267351
71,2007,N15,서울역버스환승센터.강우규의거터,126.972971,37.555614
72,2007,N75,경기도동물위생시험소,126.940002,37.267351
73,2007,N75,서울역버스환승센터.강우규의거터,126.972971,37.555614
337,5220,N61,신흥지구대,127.143659,37.441736
338,5220,N61,신흥지구대,127.143707,37.441750
393,6026,N13,성일고.힐스테이트아파트,127.147456,37.435513
394,6026,N13,용두동사거리.한국의류시험연구원,127.028388,37.577146
395,6026,N26,성일고.힐스테이트아파트,127.147456,37.435513
396,6026,N26,용두동사거리.한국의류시험연구원,127.028388,37.577146
